In [1]:
import numpy as np

In [2]:
import tensorflow as tf

2024-07-21 09:11:02.008986: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-21 09:11:02.010607: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-21 09:11:02.015398: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-21 09:11:02.031983: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 09:11:02.066084: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

In [3]:
input_dir = "../data/"
skeleton_loc = input_dir + "0002-L_Skeleton.txt"
label_loc = input_dir + "0002-L_Label.txt"

In [4]:
with open(skeleton_loc) as f:
    skeletons = f.read().split("\n")

skeletons = [s.split(" ")[:25*3] for s in skeletons]
_ = skeletons.pop()

In [5]:
sk_np = np.array([
    [list(map(float, s[j*3 : (j+1)*3])) for j in range(len(s) // 3)] for s in skeletons
])

In [6]:
frames = sk_np.shape[0]

In [7]:
# Interpolate

sk_np = tf.image.resize(sk_np, size=(2048, sk_np.shape[1]), method=tf.image.ResizeMethod.BILINEAR).numpy()

In [8]:
# VIT

for t in sk_np:
    t -= t[0]

In [9]:
# TSI

maxes = np.max(sk_np, axis=(0, 1))
mins = np.min(sk_np, axis=(0, 1))
max_diff = np.max(maxes - mins)

for c in range(sk_np.shape[-1]):
    sk_np[:,:,c] = np.floor(255 * (sk_np[:,:,c] - mins[c]) / max_diff) / 255

In [10]:
sk_image = tf.constant(sk_np)

In [11]:
sk_motion = sk_image[1:,:,:] - sk_image[:-1,:,:]
sk_motion = tf.image.resize(sk_motion, size=(2048, sk_motion.shape[1]), method=tf.image.ResizeMethod.BILINEAR)

In [12]:
sk_image.shape

TensorShape([2048, 25, 3])

In [14]:
sk_motion.shape

TensorShape([2048, 25, 3])

In [15]:
with open(label_loc) as f:
    labels = f.read().split("\n")
    
labels = [list(map(int, l.split(",")[:-1])) for l in labels]
_ = labels.pop()

labels = list(filter(lambda x: x[0] <= 10, labels))

In [16]:
def map_point(x, frames):
    return x * 2048 / frames

In [23]:
heatmap_t = np.zeros((1024, 1, 10))
offset_t = np.zeros((1024, 1, 1))
size_t = np.zeros((1024, 1, 1))

for label in labels:
    cat = label[0] - 1
    c = (label[2] + label[1]) / 2

    s = np.floor(map_point(label[1], frames))
    e = np.ceil(map_point(label[2], frames))
    c = np.floor(map_point(c, frames))

    off = (s / 2) - np.floor(s / 2)

    s = int(np.floor(s / 2) - 1)
    e = int(np.floor(e / 2) - 1)
    c = int(np.floor(c / 2) - 1)
    sig = (e - s) / 6
    
    for x, h in enumerate(heatmap_t):
        h[0][cat] += np.exp( - (x - c)**2 / (2 * sig**2))

    offset_t[c][0][0] = off
    size_t[c][0][0] = np.log(e - s)

3 99 105 112 0.0 2.5649493574615367
8 485 488 490 0.5 1.6094379124341003
7 657 669 681 0.5 3.1780538303479453
9 705 720 736 0.0 3.4339872044851463


In [34]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

print(size_t[:,0,0])

np.set_printoptions(**opt)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         